<a href="https://colab.research.google.com/github/sabari7251/Fashion-Minst/blob/main/fashion_minst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import tensorflow as tf
print(tf.__version__)

2.19.0


In [4]:
import gzip
import numpy as np

def load_images(path):
    with gzip.open(path, 'rb') as f:
        _ = int.from_bytes(f.read(4), 'big')
        num_images = int.from_bytes(f.read(4), 'big')
        rows = int.from_bytes(f.read(4), 'big')
        cols = int.from_bytes(f.read(4), 'big')

        buffer = f.read()
        data = np.frombuffer(buffer, dtype=np.uint8)
        data = data.reshape(num_images, rows, cols)
        return data

def load_labels(path):
    with gzip.open(path, 'rb') as f:
        _ = int.from_bytes(f.read(4), 'big')
        num_labels = int.from_bytes(f.read(4), 'big')

        buffer = f.read()
        labels = np.frombuffer(buffer, dtype=np.uint8)
        return labels

In [5]:
base_path = "/content/drive/MyDrive/Colab Notebooks/fashion/data/"
X_train = load_images(base_path + "train-images-idx3-ubyte.gz")
y_train = load_labels(base_path + "train-labels-idx1-ubyte.gz")

X_test = load_images(base_path + "t10k-images-idx3-ubyte.gz")
y_test = load_labels(base_path + "t10k-labels-idx1-ubyte.gz")

print("Train shape:", X_train.shape)
print("Train labels:", y_train.shape)
print("Test shape:", X_test.shape)
print("Test labels:", y_test.shape)


Train shape: (60000, 28, 28)
Train labels: (60000,)
Test shape: (10000, 28, 28)
Test labels: (10000,)


In [6]:
X_train = X_train.astype(np.float32)
X_train/=255.0
X_test = X_test.astype(np.float32)
X_test/=255.0
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

In [7]:
img=X_train[0]
flatten_img = img.flatten()
hist,bins=np.histogram(flatten_img,bins=16,range=(0,1))

In [8]:
features=[]

for img in X_train:
  flatten_img=img.flatten()
  hist,bins=np.histogram(flatten_img,bins=16,range=(0,1))
  features.append(hist)
features = np.array(features)

In [9]:
test_features=[]

for img in X_test:
  flatten_img = img.flatten()
  hist,bins=np.histogram(flatten_img,bins=16,range=(0,1))
  test_features.append(hist)
test_features = np.array(test_features)

print("Train Feature Shape:", features.shape)
print("Test Feature Shape:", test_features.shape)

Train Feature Shape: (60000, 16)
Test Feature Shape: (10000, 16)


In [10]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

nb = GaussianNB()
nb.fit(features, y_train)

y_pred_nb = nb.predict(test_features)

print("Naïve Bayes Accuracy (Histogram):",
      accuracy_score(y_test, y_pred_nb))

Naïve Bayes Accuracy (Histogram): 0.3322


In [11]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(features, y_train)

y_pred_rf = rf.predict(test_features)

print("Random Forest Accuracy (Histogram):",
      accuracy_score(y_test, y_pred_rf))

Random Forest Accuracy (Histogram): 0.4927


In [12]:
import cv2

def extract_gradient(images):
    grad_features = []
    for img in images:
        sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)
        sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)
        gradient = np.sqrt(sobelx**2 + sobely**2)
        grad_features.append(np.mean(gradient))
    return np.array(grad_features).reshape(-1,1)

X_train_grad = extract_gradient(X_train)
X_test_grad = extract_gradient(X_test)

print("Gradient Train Shape:", X_train_grad.shape)
print("Gradient Test Shape:", X_test_grad.shape)

Gradient Train Shape: (60000, 1)
Gradient Test Shape: (10000, 1)


In [13]:
nb_grad = GaussianNB()
nb_grad.fit(X_train_grad, y_train)

y_pred_nb_grad = nb_grad.predict(X_test_grad)

print("Naïve Bayes Accuracy (Gradient):",
      accuracy_score(y_test, y_pred_nb_grad))

Naïve Bayes Accuracy (Gradient): 0.2065


In [14]:
rf_grad = RandomForestClassifier(n_estimators=100, random_state=42)
rf_grad.fit(X_train_grad, y_train)

y_pred_rf_grad = rf_grad.predict(X_test_grad)

print("Random Forest Accuracy (Gradient):",
      accuracy_score(y_test, y_pred_rf_grad))

Random Forest Accuracy (Gradient): 0.1391


In [16]:
print("\nClassification Report (Random Forest - Histogram):\n")
print(classification_report(y_test, y_pred_rf))

print("\nClassification Report (Naive Bayes - Histogram)\n")
print(classification_report(y_test, y_pred_nb))


Classification Report (Random Forest - Histogram):

              precision    recall  f1-score   support

         0.0       0.46      0.49      0.47      1000
         1.0       0.62      0.68      0.65      1000
         2.0       0.45      0.51      0.48      1000
         3.0       0.42      0.41      0.42      1000
         4.0       0.41      0.41      0.41      1000
         5.0       0.67      0.63      0.65      1000
         6.0       0.37      0.32      0.34      1000
         7.0       0.59      0.60      0.60      1000
         8.0       0.43      0.30      0.35      1000
         9.0       0.48      0.58      0.52      1000

    accuracy                           0.49     10000
   macro avg       0.49      0.49      0.49     10000
weighted avg       0.49      0.49      0.49     10000


Classification Report (Naive Bayes - Histogram)

              precision    recall  f1-score   support

         0.0       0.27      0.09      0.13      1000
         1.0       0.47      